## Read Data From SQL Pool


In [9]:
val df_City = spark.read.sqlanalytics("SQLPool01.wwi.DimCity") 
val df_FactOrder = spark.read.sqlanalytics("SQLPool01.wwi.FactOrder")
 // df.show(10)

df_City: org.apache.spark.sql.DataFrame = [CityKey: int, WWICityID: int ... 11 more fields]
df_FactOrder: org.apache.spark.sql.DataFrame = [OrderKey: bigint, CityKey: int ... 17 more fields]

In [14]:
df_City.createOrReplaceTempView("city")
df_FactOrder.createOrReplaceTempView("factOrder")

## Data Exploration and Analysis


In [27]:
val df = spark.sql("select * from factOrder")

df: org.apache.spark.sql.DataFrame = [OrderKey: bigint, CityKey: int ... 17 more fields]

## Use of Magic commands


In [26]:
%%sql 
SELECT * FROM city

## Join Data


In [38]:
val df = (spark.sql("""SELECT city.StateProvince, SUM(Quantity) as TotalOrders 
                    FROM factorder 
                    JOIN  city on city.CityKey=factorder.CityKey 
                    group by city.StateProvince 
                    order by SUM(Quantity) desc """))
display(df.limit(10))

## Write result to ADLS Gen2 in Parquet format


In [51]:
// Primary storage info
val account_name = "<storage account>" // fill in your primary account name
val container_name = "<container name>" // fill in your container name
val relative_path = "<folder name>" // fill in your relative folder path

val adls_path = f"abfss://$container_name@$account_name.dfs.core.windows.net/$relative_path/"
print("Primary storage account path: " + adls_path)
val parquet_path = adls_path + "OrdersByState"
df.write.mode("Overwrite").parquet(parquet_path)


account_name: String = synapsedemos
container_name: String = curateddata
relative_path: String = WWI.parquet
adls_path: String = abfss://curateddata@synapsedemos.dfs.core.windows.net/WWI.parquet/
Primary storage account path: abfss://curateddata@synapsedemos.dfs.core.windows.net/WWI.parquet/parquet_path: String = abfss://curateddata@synapsedemos.dfs.core.windows.net/WWI.parquet/OrdersByState